## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-GPT-Test')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    #print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        listOfObj[index]['datatype']
    except:
        flag = False
        
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        if len(listOfObj[index]["value"].split('/')) == 1:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]['value'])+ '"')
        else:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '<'+str(listOfObj[index]['value'])+ '>')
    #print(answer)
    
    
    if answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '".')  
            else:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')    
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '". }}') 
            else:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 25
            
            """) 
        
    return FilterQuestion

In [4]:
def pickSubject():
    #ind=randint(0,10000)
    entities = []
    query = f"""
    select distinct ?s
    where {{
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        entities.append(r['s']['value'])
        #return("<"+r["s"]["value"]+">")
    #data = pd.read_csv('abstracts.csv')
    return entities

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    #print(subject)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [5]:
def numberleft(PosFilters,NegFilters):
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-GPT-Test')
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [6]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [7]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [8]:
import pandas as pd 
data1 = pd.read_csv('Abstracts-186.csv')
data1 = data1.drop('Unnamed: 0', axis=1)

data1

,Entity,Label,Abstract
0,http://yago-knowledge.org/resource/David_Bowie,David Bowie,David Robert Jones (8 January 1947 – 10 Januar...
1,http://yago-knowledge.org/resource/Kanye_West,Kanye West,Ye (/jeɪ/ YAY; born Kanye Omari West /ˈkɑːnjeɪ...
2,http://yago-knowledge.org/resource/Eminem,Eminem,"Marshall Bruce Mathers III (born October 17, 1..."
3,http://yago-knowledge.org/resource/Jennifer_An...,Jennifer Aniston,"Jennifer Joanna Aniston (born February 11, 196..."
4,http://yago-knowledge.org/resource/George_W._Bush,George W. Bush,"George Walker Bush (born July 6, 1946) is an A..."
...,...,...,...
179,http://yago-knowledge.org/resource/Nancy_Pelosi,Nancy Pelosi,Nancy Patricia Pelosi (/pəˈloʊsi/; née D'Alesa...
180,http://yago-knowledge.org/resource/Andhra_Pradesh,Andhra Pradesh,"Andhra Pradesh (English: /ˌɑːndrə prəˈdɛʃ/, Te..."
181,http://yago-knowledge.org/resource/Maharashtra,Maharashtra,Maharashtra (/məhɑːˈrɑːʃtrə/; Marathi: [məhara...
182,http://yago-knowledge.org/resource/Paris_Saint...,Paris Saint-Germain F.C.,Paris Saint-Germain Football Club (French pron...


In [9]:
import random
import pandas as pd
random.seed(10)
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
subjects = data1['Entity'][gamesplayed:]
totalgames = len(subjects)
results = []

In [10]:
while gamesplayed < totalgames:
    # Resets all variables
    try:
        left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
        i=0
        outofguess = []
        #print(gamesplayed)
        # pick subject
        subject = "<" + subjects[gamesplayed] + ">"
        #print(subject)

        while i < 20:

            # list of Pred and Obj for questions
            listOfObj ,listOfPred = extractCountAndAttr(qres)

            # if only 1 popular entity is left break
            if left<=1:
                break

            # ask question and add filters to query
            #print('Question: %s' %(i+1))
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

            # number of popular entities left
            left = numberleft(PosFilters, NegFilters)
            #print('left -',left)

            # run query
            sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-GPT-Test')
            query = updateQuery(left/2,PosFilters, NegFilters)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            qres = sparql.queryAndConvert()

            i+=1

        # Prints answer
        if i != 20:
            for attr in popentities(PosFilters, NegFilters):
                for x in [attr['s']['value']][0:10]:
                    answer = (x.split("/")[-1])
            #print("Your Answer is: " + answer)
            if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
                wincount += 1
                gamesplayed += 1
                results.append(i)
                #results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': i})
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': i})
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                results.append(i)
                #results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': i})
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': i})
                print('%s/%s' %(wincount,gamesplayed))
                print()

        # ran out of questions
        else:
            #print('Final Question:')
            for j in popentities(PosFilters, NegFilters):
                    outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
            guess = random.choice(outofguess)
            #print("Is it " + guess + "?")
            if str(subject.split("/")[-1]) == str(guess + ">"):
                wincount += 1
                gamesplayed += 1
                #results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': 20})
                results.append(20)
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': 20})
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                #results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': 20})
                results.append(20)
                print(gamesplayed, {'Subject': subject.split("/")[-1].split('>')[0], 'Questions-C': 20})
                print('%s/%s' %(wincount,gamesplayed))
                print()
        breakskip = 0
    except KeyboardInterrupt:
        breakf
    except:
        continue
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

1 {'Subject': 'David_Bowie', 'Questions-C': 7}
1/1

2 {'Subject': 'Kanye_West', 'Questions-C': 7}
2/2

3 {'Subject': 'Eminem', 'Questions-C': 9}
3/3

4 {'Subject': 'Jennifer_Aniston', 'Questions-C': 7}
4/4

5 {'Subject': 'George_W._Bush', 'Questions-C': 8}
5/5

6 {'Subject': 'Prince_(musician)', 'Questions-C': 8}
6/6

7 {'Subject': 'Justin_Bieber', 'Questions-C': 7}
7/7

8 {'Subject': 'Drake_(musician)', 'Questions-C': 6}
8/8

9 {'Subject': 'Heath_Ledger', 'Questions-C': 7}
9/9

10 {'Subject': 'John_Krasinski', 'Questions-C': 7}
10/10

11 {'Subject': 'Val_Kilmer', 'Questions-C': 7}
11/11

12 {'Subject': 'American_Horror_Story', 'Questions-C': 7}
12/12

13 {'Subject': 'Brad_Pitt', 'Questions-C': 7}
13/13

14 {'Subject': 'Japan', 'Questions-C': 7}
14/14

15 {'Subject': 'Barack_Obama', 'Questions-C': 7}
15/15

16 {'Subject': 'Bill_Clinton', 'Questions-C': 7}
16/16

17 {'Subject': 'Donald_Trump', 'Questions-C': 8}
17/17

18 {'Subject': 'Jimmy_Carter', 'Questions-C': 8}
18/18

19 {'Subject'

In [14]:
# if literal is less than 2-3 words make a link version
import pandas as pd 
thesis_results = pd.read_csv('ThesisResults.csv')
thesis_results = thesis_results.drop('Unnamed: 0', axis=1)

thesis_results

,Subject,Questions-C,Questions-Base,Questions-S,Questions-GPT
0,David_Bowie,7,7,7,7
1,Kanye_West,6,6,7,7
2,Eminem,7,8,8,9
3,Jennifer_Aniston,7,7,7,7
4,George_W._Bush,7,7,7,7
...,...,...,...,...,...
179,Nancy_Pelosi,8,8,8,8
180,Andhra_Pradesh,8,8,8,8
181,Maharashtra,8,8,7,8
182,Paris_Saint-Germain_F.C.,9,9,8,9


In [12]:
#9/21 - ClauCy
#11/21 - Stanford
#thesis_results = pd.DataFrame(results)
thesis_results['Questions-GPT2'] = results
thesis_results
#thesis_results.to_csv('ThesisResults.csv')

,Subject,Questions-C,Questions-Base,Questions-S,Questions-GPT,Questions-GPT2
0,David_Bowie,7,7,7,7,7
1,Kanye_West,6,6,7,7,7
2,Eminem,7,8,8,9,9
3,Jennifer_Aniston,7,7,7,7,7
4,George_W._Bush,7,7,7,7,8
...,...,...,...,...,...,...
179,Nancy_Pelosi,8,8,8,8,8
180,Andhra_Pradesh,8,8,8,8,8
181,Maharashtra,8,8,7,8,8
182,Paris_Saint-Germain_F.C.,9,9,8,9,9


In [5]:
import pandas as pd 
data1 = pd.read_csv('ThesisResults.csv')
data1 = data1.drop('Unnamed: 0', axis=1)
#data1 = data1.drop('Questions-GPT2', axis=1)
#data1.to_csv('betterThesisResults.csv')
data1
#data1.T.to_csv('flippedata.csv')

,Subject,Questions-C,Questions-Base,Questions-S,Questions-GPT
0,David_Bowie,7,7,7,7
1,Kanye_West,6,6,7,7
2,Eminem,7,8,8,9
3,Jennifer_Aniston,7,7,7,7
4,George_W._Bush,7,7,7,7
...,...,...,...,...,...
179,Nancy_Pelosi,8,8,8,8
180,Andhra_Pradesh,8,8,8,8
181,Maharashtra,8,8,7,8
182,Paris_Saint-Germain_F.C.,9,9,8,9


In [9]:
datas = []
for s,c,b,st,g in data1.values:
    datas.append({'Model': 'Base', 'Subject': s, 'Questions': b})
    datas.append({'Model': 'ClauCy', 'Subject': s, 'Questions': c})
    datas.append({'Model': 'Stanford', 'Subject': s, 'Questions': st})
    datas.append({'Model': 'GPT', 'Subject': s, 'Questions': g})
    
d = pd.DataFrame(datas)
d.to_csv('thesisdata3.csv')

In [12]:
clcy = 0
stan = 0
gpt = 0
base = 0

for _,c,b,s,g in data1.values:
    clcy += c
    stan += s
    gpt += g
    base += b

base,clcy,stan,gpt
    

(1446, 1431, 1426, 1425)

In [ ]:
basetie = 0
base = 0
gptbase = 0

clcytie = 0
clcy = 0
gptclcy = 0

stantie = 0
stan = 0
gptstan = 0

basetie2 = 0
base2 = 0
gptbase2 = 0

clcytie2 = 0
clcy2 = 0
gptclcy2 = 0

stantie2 = 0
stan2 = 0
gptstan2 = 0

gpt2score = 0
gptscore = 0
gpttie = 0


for _,c,b,s,gpt,gpt2 in thesis_results.values:
    if b == gpt:
        basetie += 1
    elif gpt > b:
        base += 1
    else:
        gptbase += 1
        
    if c == gpt:
        clcytie += 1
    elif gpt > c:
        clcy += 1
    else:
        gptclcy += 1
        
    if s == gpt:
        stantie += 1
    elif gpt > s:
        stan += 1
    else:
        gptstan += 1
    
    if b == gpt2:
        basetie2 += 1
    elif gpt2 > b:
        base2 += 1
    else:
        gptbase2 += 1
        
    if c == gpt2:
        clcytie2 += 1
    elif gpt2 > c:
        clcy2 += 1
    else:
        gptclcy2 += 1
        
    if s == gpt2:
        stantie2 += 1
    elif gpt2 > s:
        stan2 += 1
    else:
        gptstan2 += 1
        
    if gpt == gpt2:
        gpttie += 1
    elif gpt2 > gpt:
        gptscore += 1
    else:
        gpt2score += 1
        
print('GPT-Base:')
print('gpt score:',gptbase,'-base score:', base,'-tie score:', basetie) 
print('gpt %:', round((gptbase/(gptbase+base+basetie))*100,2))
print('base %:', round((base/(gptbase+base+basetie))*100,2))
print('tie %:', round((basetie/(gptbase+base+basetie))*100,2))
if gptbase > base:
    print('GPT best')
else:
    print('Not best')
print()
print('GPT-ClCy:')
print('gpt score:',gptclcy,'-ClCy score:', clcy,'-tie score:', clcytie) 
print('gpt %:', round((gptclcy/(gptclcy+clcy+clcytie))*100,2))
print('clcy %:', round((clcy/(gptclcy+clcy+clcytie))*100,2))
print('tie %:', round((clcytie/(gptclcy+clcy+clcytie))*100,2))
if gptclcy > clcy:
    print('GPT best')
else:
    print('Not best')
print()
print('GPT-Stanford:')
print('gpt score:',gptstan,'-Stanford score:', stan,'-tie score:', stantie) 
print('gpt %:', round((gptstan/(gptstan+stan+stantie))*100,2))
print('stan %:', round((stan/(gptstan+stan+stantie))*100,2))
print('tie %:', round((stantie/(gptstan+stan+stantie))*100,2))
if gptstan > stan:
    print('GPT best')
else:
    print('Not best')
print()
print()
print('GPT2-Base:')
print('gpt2 score:',gptbase2,'-base score:', base2,'-tie score:', basetie2) 
print('gpt2 %:', round((gptbase2/(gptbase2+base2+basetie2))*100,2))
print('base %:', round((base2/(gptbase2+base2+basetie2))*100,2))
print('tie %:', round((basetie2/(gptbase2+base2+basetie2))*100,2))
if gptbase2 > base2:
    print('GPT2 best')
else:
    print('Not best')
print()
print('GPT2-ClCy:')
print('gpt2 score:',gptclcy2,'-ClCy score:', clcy2,'-tie score:', clcytie2) 
print('gpt2 %:', round((gptclcy2/(gptclcy2+clcy2+clcytie2))*100,2))
print('clcy %:', round((clcy2/(gptclcy2+clcy2+clcytie2))*100,2))
print('tie %:', round((clcytie2/(gptclcy2+clcy2+clcytie2))*100,2))
if gptclcy2 > clcy2:
    print('GPT2 best')
else:
    print('Not best')
print()
print('GPT2-Stanford:')
print('gpt2 score:',gptstan2,'-Stanford score:', stan2,'-tie score:', stantie2) 
print('gpt2 %:', round((gptstan2/(gptstan2+stan2+stantie2))*100,2))
print('stan %:', round((stan/(gptstan2+stan2+stantie2))*100,2))
print('tie %:', round((stantie/(gptstan2+stan2+stantie2))*100,2))
if gptstan2 > stan2:
    print('GPT2 best')
else:
    print('Not best')
print()
print()
print()
print('GPT2-GPT:')
print('gpt2 score:',gpt2score,'-Stanford score:', gptscore,'-tie score:', gpttie) 
print('gpt2 %:', round((gpt2score/(gpt2score+gptscore+gpttie))*100,2))
print('gpt %:', round((gptscore/(gpt2score+gptscore+gpttie))*100,2))
print('tie %:', round((gpttie/(gpt2score+gptscore+gpttie))*100,2))
if gpt2score > gptscore:
    print('GPT2 best')
else:
    print('GPT best')
# MAKE THIS A FUNCTION (WAY TOO LONG)

In [15]:
def results( results1, results2, label1, label2 ):
    score1 = 0
    score2 = 0
    tie = 0
    for res1,res2 in zip( results1, results2 ):
        if res1 == res2:
            tie += 1
        elif res1 > res2:
            score2 += 1
        else:
            score1 += 1
    
    print( '%s-%s:' %( label1.upper(), label2.upper() ) )
    print( '%s score:' %( label1 ), score1, '-%s score:' %( label2 ), score2, '-tie score:', tie ) 
    print( '%s:' %(label1), round( ( score1/( score1+score2+tie ) ) *100,2 ) )
    print( '%s:' %(label2), round( ( score2/( score1+score2+tie ) ) *100,2 ) )
    print( 'tie %:', round( ( tie/( score1+score2+tie ) ) *100,2 ) )
    if score1 > score2:
        print( '%s best' %( label1.upper() ) )
    else:
        print( '%s best' %( label2.upper() ) )
    #return [score1,score2,tie]
    return

results( thesis_results['Questions-GPT'], thesis_results['Questions-Base'], 'gpt', 'base' )
print()
results( thesis_results['Questions-GPT'], thesis_results['Questions-C'], 'gpt', 'clcy' ) 
print()
results( thesis_results['Questions-GPT'], thesis_results['Questions-S'], 'gpt', 'stan' )
print()
results( thesis_results['Questions-C'], thesis_results['Questions-Base'], 'clcy', 'base' )
print()
results( thesis_results['Questions-S'], thesis_results['Questions-Base'], 'stan', 'base' )
print()
results( thesis_results['Questions-S'], thesis_results['Questions-C'], 'stan', 'clcy' )

GPT-BASE:
gpt score: 49 -base score: 39 -tie score: 96
gpt: 26.63
base: 21.2
tie %: 52.17
GPT best

GPT-CLCY:
gpt score: 52 -clcy score: 46 -tie score: 86
gpt: 28.26
clcy: 25.0
tie %: 46.74
GPT best

GPT-STAN:
gpt score: 50 -stan score: 45 -tie score: 89
gpt: 27.17
stan: 24.46
tie %: 48.37
GPT best

CLCY-BASE:
clcy score: 20 -base score: 22 -tie score: 142
clcy: 10.87
base: 11.96
tie %: 77.17
BASE best

STAN-BASE:
stan score: 29 -base score: 26 -tie score: 129
stan: 15.76
base: 14.13
tie %: 70.11
STAN best

STAN-CLCY:
stan score: 36 -clcy score: 33 -tie score: 115
stan: 19.57
clcy: 17.93
tie %: 62.5
STAN best


In [33]:
s = 0
g = 0
c = 0
tie = 0
gs_tie = 0
gc_tie = 0
sc_tie = 0

for _,clcy,base,stan,gpt in thesis_results.values:
    if stan == gpt and gpt == clcy:
        #s += 1
        #g += 1
        #c += 1
        tie += 1 
    elif stan == gpt and stan < clcy:
        #s += 1
        #g += 1
        gs_tie += 1 
    elif stan == clcy and gpt > clcy:
        #s += 1
        #c += 1
        #print(stan,clcy,gpt)
        sc_tie += 1
    elif gpt == clcy and gpt < stan:
        #g += 1
        #c += 1
        gc_tie += 1
    elif gpt < stan and gpt < clcy:
        g += 1
    elif stan < gpt and stan < clcy:
        s += 1 
    elif clcy < gpt and clcy < stan:
        c += 1 
print('clcy:',c,'- Stan:',s,'- GPT:',g,'- tie:',tie,'- gptstan-tie:',gs_tie,'- gptclcy-tie:',gc_tie,'- stanclcy:',sc_tie) 
print(c+tie+gc_tie+sc_tie)
print(s+tie+gs_tie+sc_tie)
print(g+tie+gs_tie+gc_tie)
print()
print(round(20/(20+18+35+62+13+12+24)*100,1))
print(round(18/(20+18+35+62+13+12+24)*100,1))
print(round(35/(20+18+35+62+13+12+24)*100,1))

clcy: 20 - Stan: 18 - GPT: 35 - tie: 62 - gptstan-tie: 13 - gptclcy-tie: 12 - stanclcy: 24
118
117
122

10.9
9.8
19.0


In [ ]:
sub = []
for i in subjects_lost:
    i = i.split('<')[1]
    i = i.split('>')[0]
    sub.append("%s" %(i))
sub

In [ ]:
import random
import pandas as pd
random.seed(10)
#subjects = pickSubject()
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
#totalgames = len(subjects)

left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
outofguess = []
subject = "<http://yago-knowledge.org/resource/Jesse_Metcalfe>"
print(subject)

while i < 20:

    # list of Pred and Obj for questions
    listOfObj ,listOfPred = extractCountAndAttr(qres)

    # if only 1 popular entity is left break
    if left<=1:
        break

    # ask question and add filters to query
    #print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

    # number of popular entities left
    left = numberleft(PosFilters, NegFilters)

    # run query
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-GPT-Test')
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.queryAndConvert()

    i+=1

# Prints answer
if i != 20:
    for attr in popentities(PosFilters, NegFilters):
        for x in [attr['s']['value']][0:10]:
            answer = (x.split("/")[-1])
    print("Your Answer is: " + answer)
    if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
        wincount += 1
        gamesplayed += 1
        #print("Win!")
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print("Lose...")
        print('%s/%s' %(wincount,gamesplayed))
        print()

# ran out of questions
else:
    print('Final Question:')
    for j in popentities(PosFilters, NegFilters):
            outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
    guess = random.choice(outofguess)
    print("Is it " + guess + "?")
    if str(subject.split("/")[-1]) == str(guess + ">"):
        wincount += 1
        gamesplayed += 1
        #print('Win!')
        #print('%s/%s' %(wincount,gamesplayed))
        #print()
    else:
        gamesplayed += 1
        subjects_lost.append(subject)
        print('Lose...')
        print('%s/%s' %(wincount,gamesplayed))
        print()
breakskip = 0
   
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

#### Seeds used:
- 10
- ??
- ??

#### Time started:
- 10
    - base model
        - 22:50


In [ ]:
len('ALL'.split('/'))